In [1]:
from transformers import GPT2LMHeadModel

In [15]:
model_hf = GPT2LMHeadModel.from_pretrained('gpt2')

model_hf_sd = model_hf.state_dict()
for k, v in model_hf_sd.items():
    print(k,"----------------------------------" ,v.shape)

transformer.wte.weight ---------------------------------- torch.Size([50257, 768])
transformer.wpe.weight ---------------------------------- torch.Size([1024, 768])
transformer.h.0.ln_1.weight ---------------------------------- torch.Size([768])
transformer.h.0.ln_1.bias ---------------------------------- torch.Size([768])
transformer.h.0.attn.c_attn.weight ---------------------------------- torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias ---------------------------------- torch.Size([2304])
transformer.h.0.attn.c_proj.weight ---------------------------------- torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias ---------------------------------- torch.Size([768])
transformer.h.0.ln_2.weight ---------------------------------- torch.Size([768])
transformer.h.0.ln_2.bias ---------------------------------- torch.Size([768])
transformer.h.0.mlp.c_fc.weight ---------------------------------- torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias ----------------------------------

In [12]:
from tradeformer.model import TradeFormer, TradeFormerConfig
from tradeformer.tokenizer import MarketTokenizer


with open('./Data/preprocessed_data/train.txt', 'r') as f:
    train_data = f.readlines()


cfg = TradeFormerConfig()
model = TradeFormer(cfg)
tokenizer = MarketTokenizer('./trained_tokenizer/')

In [4]:
import torch

test_inp = torch.randint(0, 100, (5, 8), dtype=torch.long)
model(test_inp).shape

torch.Size([5, 8, 1000])

In [18]:
test_inp_str = [(' '.join(x.split()[:10])) for x in train_data[0:5]]
test_inp_toknized = torch.tensor([tokenizer.encode(test_inp_str[i]) for i in range(len(test_inp_str))])
test_inp_str, test_inp_toknized

(['AJM AIK AEO BGO BIM AHN BGQ AHO BJO BJM',
  'AJM AIK AEO BGO BIM AHN BGQ AHO BJO BJM',
  'AJM AIK AEO BGO BIM AHN BGQ AHO BJO BJM',
  'AJM AIK AEO BGO BIM AHN BGQ AHO BJO BJM',
  'AJM AIK AEO BGO BIM AHN BGQ AHO BJO BJM'],
 tensor([[ 44,  35, 171,  86, 785, 166,  93],
         [ 44,  35, 171,  86, 785, 166,  93],
         [ 44,  35, 171,  86, 785, 166,  93],
         [ 44,  35, 171,  86, 785, 166,  93],
         [ 44,  35, 171,  86, 785, 166,  93]]))

In [19]:
model(test_inp_toknized)

tensor([[[-0.0732, -0.6493, -0.5046,  ..., -0.3713,  0.5752, -0.0057],
         [-0.1772, -0.1031, -0.2421,  ..., -0.9707, -0.2175,  0.1517],
         [-0.5231,  0.6064,  0.1934,  ..., -0.3642, -0.0319,  0.6557],
         ...,
         [ 0.1375, -0.3910, -0.8236,  ..., -0.2809, -0.2844, -0.7719],
         [-0.8966,  0.7704, -0.5022,  ..., -0.4647, -1.1333,  0.4188],
         [ 1.1706, -0.0215,  0.0896,  ..., -0.1421, -0.0074, -0.3911]],

        [[-0.0732, -0.6493, -0.5046,  ..., -0.3713,  0.5752, -0.0057],
         [-0.1772, -0.1031, -0.2421,  ..., -0.9707, -0.2175,  0.1517],
         [-0.5231,  0.6064,  0.1934,  ..., -0.3642, -0.0319,  0.6557],
         ...,
         [ 0.1375, -0.3910, -0.8236,  ..., -0.2809, -0.2844, -0.7719],
         [-0.8966,  0.7704, -0.5022,  ..., -0.4647, -1.1333,  0.4188],
         [ 1.1706, -0.0215,  0.0896,  ..., -0.1421, -0.0074, -0.3911]],

        [[-0.0732, -0.6493, -0.5046,  ..., -0.3713,  0.5752, -0.0057],
         [-0.1772, -0.1031, -0.2421,  ..., -0

In [23]:
import torch.nn.functional as F
#TODO move the generate function into the model class
def generate(tokens, len=5, max_len=20, num_top_samples=10, device='cpu'):
    # TOKENS: B, T
    tokens = tokens.to(device) 
    while tokens.size(1) < max_len:
        with torch.no_grad():
            out = model(tokens)

            logits = out[:, -1, :]
            probs = F.softmax(logits, 1)

            topk_probs, topk_idx = torch.topk(probs, num_top_samples, dim=-1)

            ix = torch.multinomial(topk_probs, 1)
            xcol = torch.gather(topk_idx, -1, ix) #(B, 1)

            tokens = torch.cat((tokens, xcol), dim=1)

    return tokens

out = generate(test_inp_toknized)


In [24]:
test_inp_toknized.shape,  out.shape

(torch.Size([5, 7]), torch.Size([5, 20]))

In [30]:
tokenizer.decode(out.tolist()[0])

'AJM AIK AEO BGO BIM AHN BGQ AHO BJO BJM AGO AJO BEL AHL AGL AHO AJO AGQ CEM AJO AJL BDL BEL BGM BJO AEM AHL BHN BFL AFL BFO AHL BGL BEM BEO AIO'